### Hedging Unleaded Gasoline

The analysis pertains to hedging unleaded gasoline using futures contracts traded on the NY Mercantile Exchange (NYMEX) delivered to NY harbor. The goal is to find the minimum-variance hedge ratios for gasoline alone, cross-hedged against crude oil, and cross-hedged against heating oil. The effectiveness of the hedge is evaluated from January 2024 to February 23, 2024.

Finding the minimum-variance hedge ratios, $\beta_{SF}$, for gasoline alone, cross-hedged against crude oil, and cross hedged against heating oil by running the following regressions:

$$\Delta S = \alpha + \beta_{SF} \Delta F + \varepsilon$$

In [2]:
import pandas as pd
from statsmodels.regression.linear_model import OLS
import warnings; warnings.simplefilter(action='ignore', category=FutureWarning)
import statsmodels.api as sm

futures_file_path = 'PET_PRI_FUT_S1_D.xls'
spot_file_path = 'PET_PRI_SPT_S1_D.xls'

def read_commodity_data(file_path):
    xls = pd.ExcelFile(file_path)
    dfs = {}
    for sheet_name in xls.sheet_names:
        if sheet_name != 'Contents':
            df = pd.read_excel(xls, sheet_name=sheet_name, skiprows=2)
            commodity_name = pd.read_excel(xls, sheet_name=sheet_name).columns[1]
            # df['Commodity Name'] = commodity_name
            df = df[df['Date'] >= pd.Timestamp('2013-01-01')]
            dfs[sheet_name] = df
    return dfs

futures_dfs = read_commodity_data(futures_file_path)
spot_dfs = read_commodity_data(spot_file_path)

rbob_spot = spot_dfs['Data 3']
spot_rbob_col = "Los Angeles Reformulated RBOB Regular Gasoline Spot Price (Dollars per Gallon)"
rbob_spot['Change in RBOB Spot'] = rbob_spot[spot_rbob_col].diff()

rbob_futures = futures_dfs['Data 3']
futures_rbob_col = "New York Harbor Reformulated RBOB Regular Gasoline Future Contract 1 (Dollars per Gallon)"
rbob_futures['Change in RBOB Futures'] = rbob_futures[futures_rbob_col].diff()

crude_futures = futures_dfs['Data 1']
futures_crude_col = "Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)"
crude_futures[futures_crude_col] = crude_futures[futures_crude_col] / 42  # making the futures price per gallon
crude_futures['Change in Crude Futures'] = crude_futures[futures_crude_col].diff()

heating_futures = futures_dfs['Data 4']
futures_heating_col = "New York Harbor No. 2 Heating Oil Future Contract 1 (Dollars per Gallon)"
heating_futures['Change in Heating Futures'] = heating_futures[futures_heating_col].diff()

futures_cols = {'RBOB': futures_rbob_col, 'Crude': futures_crude_col, 'Heating': futures_heating_col}
futures_change_cols = {'RBOB': 'Change in RBOB Futures', 'Crude': 'Change in Crude Futures', 'Heating': 'Change in Heating Futures'}


full_data = pd.merge(rbob_spot, rbob_futures, on='Date').merge(crude_futures, on='Date').merge(heating_futures, on='Date')[['Date', spot_rbob_col, futures_rbob_col, futures_crude_col, futures_heating_col, 'Change in RBOB Spot', 'Change in RBOB Futures', 'Change in Crude Futures', 'Change in Heating Futures']].dropna()

# data is from beginning of 2013 to end of 2023
regression_data = full_data[full_data['Date'] < pd.Timestamp('2024-01-01')]

futures_hedging_ratios = {}

# making a dataframe to store coefficient intercepts and r-squared values
coefficients_df = pd.DataFrame(columns=['Commodity', 'Intercept', 'Beta', 'R-Squared'])

for commodity in futures_change_cols:
    X = regression_data[futures_change_cols[commodity]]
    y = regression_data['Change in RBOB Spot']
    X = sm.add_constant(X)
    model = OLS(y, X)
    results = model.fit()
    optimal_hedge_ratio = round(results.params[futures_change_cols[commodity]], 5)
    futures_hedging_ratios[commodity] = optimal_hedge_ratio
    
    coefficients_df = pd.concat([coefficients_df, pd.DataFrame({'Commodity': [commodity], 'Intercept': [results.params['const']], 'Beta': [results.params[futures_change_cols[commodity]]], 'R-Squared': [results.rsquared]})])
    
coefficients_df

,Commodity,Intercept,Beta,R-Squared
0,RBOB,0.000052,0.827049,0.215827
0,Crude,0.000006,0.541912,0.087123
0,Heating,-0.000033,0.460486,0.087608


Let's assume that the company need 1,000,000 gallons of gasoline and the size of one futures contract is 42,000 gallons. We can calculate optimal number of contract to purchase to hedge the position:

$$N^* = \beta_{SF} \cdot \frac{Q_s}{Q_f}$$


Evaluating the performance of the hedge for each commodity with the following formula:

$$(S_1 - S_0) \cdot Q_s$$

Evaluating the performance of the hedge futures position for each commodity with the following formula:

$$(S_1 - F_0) \cdot (N^* \cdot Q_f)$$


In [3]:
Qa = 1000000  # Assume we want to hedge 1,000,000 gallons of gasoline
Qf = 42000  # Size of one futures contract (in gallons)

evaluation_data = full_data[full_data['Date'] >= pd.Timestamp('2024-01-01')]

s0 = evaluation_data[spot_rbob_col].iloc[0]
s1 = evaluation_data[spot_rbob_col].iloc[-1]
spot_price_change = s1 - s0
print(f"Quantity of gasoline the company needs: {Qa} gallons")
print(f"Spot price at t=0: ${s0:.2f} per gallon")
print(f"Spot price at t=1: ${s1:.2f} per gallon")
print(f"Change in spot price: ${spot_price_change:.2f} per gallon")
unhedged_performance = spot_price_change * Qa  # Change in spot price times quantity of asset
print(f"Unhedged Performance: ${unhedged_performance:.2f}\n")
print(f"Size of one futures contract: {Qf} gallons\n")

for commodity in futures_hedging_ratios:
    
    f0 = evaluation_data[futures_cols[commodity]].iloc[0]
    n_star = futures_hedging_ratios[commodity] * (Qa / Qf)
    hedged_performance = (s1 - f0) * (n_star * Qf)

    print(f"Commodity: {commodity}")
    print(f"{commodity} Futures Price at t=0: ${f0:.2f} per gallon")
    print(f"Optimal Hedge Ratio for {commodity}: {futures_hedging_ratios[commodity]}")
    print(f"Optimal number of contracts to purchase: {n_star:.2f}")
    print(f"Hedged Performance for {commodity}: ${hedged_performance:.2f}\n")

Quantity of gasoline the company needs: 1000000 gallons
Spot price at t=0: $2.45 per gallon
Spot price at t=1: $2.61 per gallon
Change in spot price: $0.16 per gallon
Unhedged Performance: $162000.00

Size of one futures contract: 42000 gallons

Commodity: RBOB
RBOB Futures Price at t=0: $2.10 per gallon
Optimal Hedge Ratio for RBOB: 0.82705
Optimal number of contracts to purchase: 19.69
Hedged Performance for RBOB: $427584.85

Commodity: Crude
Crude Futures Price at t=0: $1.68 per gallon
Optimal Hedge Ratio for Crude: 0.54191
Optimal number of contracts to purchase: 12.90
Hedged Performance for Crude: $507382.59

Commodity: Heating
Heating Futures Price at t=0: $2.53 per gallon
Optimal Hedge Ratio for Heating: 0.46049
Optimal number of contracts to purchase: 10.96
Hedged Performance for Heating: $39602.14



### Summarized Results


##### 1. Futures Position Statement:
   To hedge against the rising price of gasoline, we would undertake a long position in RBOB gasoline futures contracts, crude oil futures contracts, and heating oil futures contracts with the respective optimal hedge ratios and quantities. 
  

##### 2. Estimated Regression Equation:
Let's assume the estimated regression equation that determined the hedge is of the form:
   
$$ \Delta S = \alpha + \beta_{SF} \Delta F + \varepsilon $$

Where:

- $ \Delta S $ is the change in spot price,
- $ \alpha $ is the intercept of the regression equation,
- $ \beta_{SF} $ is the hedge ratio as the regression coefficient,
- $ \Delta F $ is the change in futures price,
- $ \varepsilon $ is the error term of the regression.

The optimal hedge ratios ($ \beta_{SF} $) for RBOB, crude, and heating oil are 0.82705, 0.54191, and 0.46049 respectively. The ratios are computed from data ranging from January 2013 to December 2023. The following was found from the regression analysis in python.
  

 

| Commodity | Regression Equation | R-Squared |
|-----------|---------------------|-----------|
| RBOB      | $\Delta S = 0.000052 + 0.827049 \Delta F$ | 0.2158    |
| Crude     | $\Delta S = 0.000006 + 0.511912 \Delta F$ | 0.0871    |
| Heating   | $\Delta S = -0.000033 + 0.460486 \Delta F$ | 0.0876    |
  


##### 3. Gain/Loss Computation:
   The gain or loss for the hedged and un-hedged positions in dollar terms are computed as follows on the evaluation period from January 2024 till February 23, 2024:

**Unhedged Position:** If no hedge was undertaken, the increase in spot prices would result in additional costs of:

$$ \text{Unhedged Loss} = \text{Change in Spot Price} \times \text{Quantity of Gasoline Needed} $$
$$ \text{Unhedged Loss} = \$0.16 \text{ per gallon} \times 1,000,000 \text{ gallons} = \$160,000 $$

**Hedged Position:** The gain or loss for each hedged position is the product of the futures price at inception, the optimal number of contracts, and the size of one futures contract, adjusted by the change in spot price.

For RBOB:


$$ \text{Hedged Gain for RBOB} = (S_1 - F_0) \times (N^* \times Q_f) $$
$$ \text{Hedged Gain for RBOB} = (\$2.61 - \$2.10) \times (19.69 \times 42,000) = \$427,584.85 $$

For Crude:
$$ \text{Hedged Gain for Crude} = (S_1 - F_0) \times (N^* \times Q_f) $$
$$ \text{Hedged Gain for Crude} = (\$2.61 - \$1.68) \times (12.90 \times 42,000) = \$507,382.59 $$

For Heating:
$$ \text{Hedged Gain for Heating} = (S_1 - F_0) \times (N^* \times Q_f) $$
$$ \text{Hedged Gain for Heating} = (\$2.61 - \$2.53) \times (10.96 \times 42,000) = \$39,602.14 $$
